# Vistral-V: A Vietnamese Large Vision-Language Model

This project demonstrates visual instruction tuning applied to the Vistral large language model (LLM). This enables the model to understand and respond to prompts that combine both visual and textual information in Vietnamese.

**Key features:**

* **Multimodal understanding:** Process images and Vietnamese text together.
* **Instruction following:** Respond to complex instructions that involve visual input.
* **Vietnamese language specialization:** Fine-tuned for accuracy and fluency in Vietnamese.

**Resources:**

* **GitHub repository:** [Vistral-V](https://github.com/hllj/Vistral-V/tree/main?tab=readme-ov-file) (Codebase and details)
* **HuggingFace model:** [Vistral-V 7B](https://huggingface.co/Vi-VLM/Vistral-V-7B) (Try it out!)


> **Access Requirement**: The following demonstration employs the [Vistral Vision model](https://huggingface.co/Vi-VLM/Vistral-V-7B), which requires authorized access. Please ensure you have the necessary permissions on Hugging Face to interact with the model before proceeding.

## Login HuggingFace

In [1]:
# Add Read HuggingFace Token to download weight models
from kaggle_secrets import UserSecretsClient
from huggingface_hub.hf_api import HfFolder

# Get HF Token, please add HF read token to Kaggle secret, or token = "hf..."
token = UserSecretsClient().get_secret("HF_TOKEN") 

HfFolder.save_token(token)

## Environment Setup

1. Git clone our repo
2. Install the customized package (which supports best for the model)
3. Install other requirements from pip

In [ ]:
# Clone the repository and install the packages of vistral-V
!git clone https://github.com/hllj/Vistral-V.git
%cd Vistral-V
!pip install -e .

# Install customized transformers
!pip install transformers==4.41.2 -q

# Install stuff
!pip install ipykernel -q
!pip install ipywidgets -q
!jupyter nbextension enable --py widgetsnbextension

fatal: destination path 'Vistral-V' already exists and is not an empty directory.
/kaggle/working/Vistral-V
Obtaining file:///kaggle/working/Vistral-V
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached tokenizers-0.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Building editable for llava (pyproject.toml) ... done
  Created wheel for llava: filename=llava-1.2.2.post1-0.editable-py3-none-any.whl size=10319 sha256=caf859bfe410954379eb1b1b8191bbb25aaf956f35b498831a59722f354a07df
  Stored in directory: /tmp/pip-ephem-wheel-cache-tg4d7f6u/wheels/03/f8/b5/2c13904

## Getting Started

This Kaggle example demonstrates how to use [Vistral Vision](https://huggingface.co/Vi-VLM/Vistral-V-7B), an AI model that can understand and discuss images in Vietnamese. 

*Note*: To run this code, you'll need access to the model on Hugging Face. If you don't have access yet, please request it before proceeding.


**What you'll see:**
* **Sets up the Vistral Vision model:** We'll load the model and get it ready to process images and text.
* **Interactive conversations:**  Ask questions about images, and Vistral Vision will respond in Vietnamese.

**Try it yourself!** Follow the steps below to run the code and start a conversation with Vistral Vision.

### Setting configs

In [ ]:
# Option 1: Lora Adapter and Base model
# model_path = "Vi-VLM/llava-vistral-7b-lora"
# model_base = "Viet-Mistral/Vistral-7B-Chat"

# Option 2: Full Model

model_path = "Vi-VLM/Vistral-V-7B"
model_base = None

conv_mode = "vistral"

image_file = "assets/images/example.jpeg"

temperature = 0.2
max_new_tokens = 512
load_8bit = False
load_4bit = False

debug = False
device = "cuda:0" # device = "cuda" if you want to inference with multiple GPU devices

### Import Libraries and Load the Model

To ensure optimal model performance and compatibility in diverse environments, we enable `disable_torch_init`. This helps prevent conflicts and improves resource management when deploying the model.

In [ ]:
from llava.utils import disable_torch_init

disable_torch_init()

In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path

# Get the model name
model_name = get_model_name_from_path(model_path)

# FIXME: Vistral-V is need to be fixed, add prefix 'llava-'
if 'vistral-v' in model_name.lower():
    model_name = 'llava-' + model_name

# Load model, tokenizer and processor stuff
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=model_base,
    model_name=model_name,
    load_8bit=load_8bit,
    load_4bit=load_4bit,
    device=device
)

### Load image using

In [ ]:
import requests
from PIL import Image
from io import BytesIO

def load_image(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

image = load_image(image_file)

print("This is the image using")
display(image)

### Configuring the Conversation Template

In [ ]:
from llava.conversation import conv_templates

if "llama-2" in model_name.lower():
    conv_mode = "llava_llama_2"
elif "mistral" in model_name.lower():
    conv_mode = "mistral_instruct"
elif "vistral" in model_name.lower():
    conv_mode = "vistral"
elif "v1.6-34b" in model_name.lower():
    conv_mode = "chatml_direct"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

conv = conv_templates[conv_mode].copy()
if "mpt" in model_name.lower():
    roles = ('user', 'assistant')
else:
    roles = conv.roles

## Let's chat !!

### Set Up Your Workspace
Start by importing the required libraries and loading the image you want to analyze.


In [ ]:
import torch

from llava.conversation import SeparatorStyle
from llava.mm_utils import process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN

from transformers import TextStreamer
from IPython.display import display

# Get the image size and díplay it
image_size = image.size

# Similar operation in model_worker.py
image_tensor = process_images([image], image_processor, model.config)
if type(image_tensor) is list:
    image_tensor = [image.to(model.device, dtype=torch.float16) for image in image_tensor]
else:
    image_tensor = image_tensor.to(model.device, dtype=torch.float16)

### Chat !

In [ ]:
while True:
    try:
        inp = input(f"{roles[0]}: ")
    except EOFError:
        inp = ""
    if not inp:
        print("exit...")
        break

    print(f"{roles[1]}: ", end="")

    if image is not None:
        # first message
        if model.config.mm_use_im_start_end:
            inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
        else:
            inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
        image = None

    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            image_sizes=[image_size],
            do_sample=True if temperature > 0 else False,
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            streamer=streamer,
            use_cache=True)

    outputs = tokenizer.decode(output_ids[0]).strip()
    conv.messages[-1][-1] = outputs

    if debug:
        print("\n", {"prompt": prompt, "outputs": outputs}, "\n")
